In [ ]:
USE_PRIVATE_DISTRO = True
DRIVE_BASE_DIR = '/content/drive/MyDrive/SMC 10/DDSP-10/' 

DRIVE_DISTRO = DRIVE_BASE_DIR + 'dist/ddsp-1.2.0.tar.gz'

if USE_PRIVATE_DISTRO:
    print("[INFO] Using private distro. Be careful.")
    from google.colab import drive
    drive.mount('/content/drive')
    !pip install -qU "$DRIVE_DISTRO"[data_preparation]
else:
    !pip install -qU ddsp[data_preparation]==1.2.0

%tensorflow_version 2.x
import tensorflow as tf

from ddsp.colab.colab_utils import play, specplot
from ddsp.synths import AmplitudeModulation
from ddsp.training import data
from ddsp import core

import glob
import numpy as np
import os
import soundfile as sf
import warnings

import seaborn as sns
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

In [ ]:
sns.set(style="whitegrid")
warnings.filterwarnings("ignore")

# AM Synth

In [ ]:
OUTPUT_FOLDER = 'am_timbre_transfer' #@param {type: "string"}

DRIVE_AUDIO_DIR = DRIVE_BASE_DIR + 'audio/' + OUTPUT_FOLDER + '/'
COLAB_AUDIO_DIR = 'data/audio/' 
AUDIO_FILEPATTERN = COLAB_AUDIO_DIR + '/*'

!mkdir -p "$DRIVE_AUDIO_DIR"
!mkdir -p "$COLAB_AUDIO_DIR"

In [ ]:
FRAME_RATE = 250
DURATION = 4

TIME_STEPS = FRAME_RATE * DURATION

MIDI_FROM = 57
MIDI_TO = 81

# Carrier amp
# car_amp = np.concatenate((np.linspace(-20,1,25), 
#                            np.ones(575), 
#                            np.linspace(1,-10,250), 
#                            -10*np.ones(150)), axis=0)
car_amp = 2*np.ones(TIME_STEPS)
car_amp[:25] = np.linspace(-20,2,25)
car_amp[-25:] = np.linspace(2,-20,25)
car_amp = car_amp[np.newaxis, :, np.newaxis]


# Modulator amp 
mod_amp = np.ones(TIME_STEPS)
mod_amp = mod_amp[np.newaxis, :, np.newaxis]

In [ ]:
for SAMPLE_RATE in [16000, 48000]:

  N_SAMPLES = SAMPLE_RATE * DURATION

  am_synth = AmplitudeModulation(n_samples=N_SAMPLES,
                                 sample_rate=SAMPLE_RATE)
  for freqs in [5, 250]:

    INSTRUMENT = 'sr{}k_mf{}'.format(SAMPLE_RATE//1000, freqs)
    DRIVE_AUDIOFILE = DRIVE_AUDIO_DIR + INSTRUMENT + '.wav'
    COLAB_AUDIOFILE = COLAB_AUDIO_DIR + INSTRUMENT + '.wav'

    print('1. Render ' + INSTRUMENT + '.wav')

    # Mod freq 
    mod_freq = freqs * np.ones(TIME_STEPS)
    mod_freq = mod_freq[np.newaxis, :, np.newaxis]

    output = []

    notes = str(MIDI_FROM) + ',' + str(MIDI_TO)

    for midinote in range(MIDI_FROM, MIDI_TO):

      # Car freq
      car_freq = core.midi_to_hz(midinote) * np.ones(TIME_STEPS)
      car_freq = car_freq[np.newaxis, :, np.newaxis]

      audio = am_synth(car_amp, car_freq, mod_amp, mod_freq)
      output = np.concatenate((output, audio[0]))

    output /= np.max(np.abs(output),axis=0)*1.1
    sf.write(DRIVE_AUDIOFILE, output, SAMPLE_RATE)
    # display(Audio(output, rate=sample_rate, autoplay=False))

    DRIVE_DATASET_DIR = DRIVE_AUDIO_DIR + INSTRUMENT + '_dataset' 
    COLAB_RECORD = 'data/' + INSTRUMENT + '_dataset/train.synthrecord' 
    RECORD_FILEPATTERN = COLAB_RECORD + '*'

    !mkdir "$DRIVE_DATASET_DIR"

    print('2. Copy {} to {}'.format(DRIVE_AUDIOFILE, COLAB_AUDIOFILE))
    !cp "$DRIVE_AUDIOFILE" "$COLAB_AUDIOFILE"

    print('3. Create ' + INSTRUMENT + ' dataset')
    if not glob.glob(COLAB_AUDIOFILE):
      raise ValueError('No audio files found')

    !ddsp_dataset_from_notes \
    --input_audio_filepatterns=$COLAB_AUDIOFILE \
    --output_tfrecord_path=$COLAB_RECORD \
    --sample_rate=$SAMPLE_RATE \
    --frame_rate=$FRAME_RATE \
    --example_secs=$DURATION \
    --midi_notes=$notes \
    --alsologtostderr

    print('4. Save to {}'.format(DRIVE_DATASET_DIR)) 
    !cp $RECORD_FILEPATTERN "$DRIVE_DATASET_DIR"

    print('\n\n\n')

In [ ]:
# data_provider = data.SynthRecordProvider(RECORD_FILEPATTERN, 
#                                          sample_rate=SAMPLE_RATE,
#                                          frame_rate=FRAME_RATE,
#                                          example_secs=DURATION)
# dataset = data_provider.get_batch(batch_size=1, shuffle=True, repeats=-1)
# dataset_iter = iter(dataset)
# for i in range (0,100):
#   frame = next(dataset_iter)
#   print(frame['f0_hz_midi'].numpy()[0][0])

# next(dataset_iter)